# Basic CRDS reference file submission

An introduction to CRDS file submission using the web submission form.

## Prerequisites

To follow along with the examples in this notebook, you will need:

- A CRDS server deployment to serve as a sandbox.  As of September 2020, the JWST B-string CRDS deployment has been commandeered to support these notebooks.

- An active STScI VPN connection.

- An account on the CRDS server with permissions to submit files.  If you need an account, contact Ed Slavich or Jonathan Eisenhamer or SCSB generally.

- The following Python packages installed:

```
$ pip install crds==7.6.0 asdf==2.7.1 astropy==4.0.1
```

## ¡CAUTION!

As you perform tasks on the CRDS website, always double-check that you are using the correct dev deployment and not one of the HST or JWST test or ops servers.  The changes you make can always be reverted, but a) the cleanup will give the CRDS operators a headache, and b) the system will retain your submissions eternally as a monument to your mistake.

That said, if you are on the correct server, please do experiment freely ("go nuts").  The dev server can be easily reset to its original state.

## Setup

We'll need to configure CRDS to use the JWST B-string server.  Let's do that:

In [ ]:
import os
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-bit.stsci.edu"
os.environ["CRDS_PATH"] = os.path.join(os.environ["HOME"], "crds-tutorial-cache")

Then import the crds client library:

In [ ]:
import crds

## Select and download an existing reference file

Now we need to identify a file to update and submit.  Ordinarily users would start from the current edit context, which is the context containing the latest updates submitted to CRDS, but here we're going to use context `jwst_0641.pmap` so that this example produces a consistent result.  Here's a link to the page for the context:

https://jwst-crds-bit.stsci.edu/context_table/jwst_0641.pmap

For this example we're going to choose a MIRI `distortion` reference file, which is one of the relatively few JWST reference files written in ASDF.  Expand the `miri` accordion and scroll down to expand `distortion`.  The page will show a table of match rules from the reference file type's **.rmap** file.  We could select a file from the table, but let's click through to the actual `jwst_miri_distortion_0031.rmap` page and view the rules directly:

```
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_miri_distortion_0030.rmap',
    'filekind' : 'DISTORTION',
    'instrument' : 'MIRI',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_miri_distortion_0031.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.INSTRUMENT.CHANNEL', 'META.INSTRUMENT.BAND', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'reference_to_dataset' : {
        'BAND' : 'META.INSTRUMENT.BAND',
        'CHANNEL' : 'META.INSTRUMENT.CHANNEL',
        'DETECTOR' : 'META.INSTRUMENT.DETECTOR',
        'EXP_TYPE' : 'META.EXPOSURE.TYPE',
    },
    'sha1sum' : '8a73a92aaa79136e054589823c0e25d79c01e25c',
}

selector = Match({
    ('ANY', 'ANY', 'ANY', 'MIR_DARK') : 'N/A',
    ('ANY', 'ANY', 'ANY', 'MIR_FLATIMAGE|MIR_FLAT-IMAGE|MIR_FLATMRS|MIR_FLAT-MRS') : 'N/A',
    ('MIRIFULONG', '34', 'LONG', 'MIR_MRS') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0032.asdf',
    }),
    ('MIRIFULONG', '34', 'MEDIUM', 'MIR_MRS') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0033.asdf',
    }),
    ('MIRIFULONG', '34', 'SHORT', 'MIR_MRS') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0034.asdf',
    }),
    ('MIRIFUSHORT', '12', 'LONG', 'MIR_MRS') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0029.asdf',
    }),
    ('MIRIFUSHORT', '12', 'MEDIUM', 'MIR_MRS') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0030.asdf',
    }),
    ('MIRIFUSHORT', '12', 'SHORT', 'MIR_MRS') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0031.asdf',
    }),
    ('MIRIMAGE', 'N/A', 'N/A', 'MIR_IMAGE|MIR_TACQ|MIR_LYOT|MIR_4QPM|MIR_CORONCAL|MIR_LRS-FIXEDSLIT|MIR_LRS-SLITLESS|MIR_TACONFIRM') : UseAfter({
        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0028.asdf',
    }),
})
```

We have several options to choose from, so let's throw a dart and work with that third match rule in the list.  That's the one that points to the file `jwst_miri_distortion_0032.asdf`.  We could just download it from the website, but why not practice using the CRDS client?

In [ ]:
result = crds.getreferences(
    {
        "META.INSTRUMENT.NAME": "MIRI",
        "META.INSTRUMENT.DETECTOR": "MIRIFULONG",
        "META.INSTRUMENT.CHANNEL": "34",
        "META.INSTRUMENT.BAND": "LONG",
        "META.EXPOSURE.TYPE": "MIR_MRS",
        "META.OBSERVATION.DATE": "2014-11-20",
        "META.OBSERVATION.TIME": "11:53:02",
    },
    reftypes=["distortion"],
    observatory="jwst",
    context="jwst_0641.pmap",
)
result

Great, now we have the file cached locally.  Let's make a working copy in our notebook directory:

In [ ]:
!cp {result["distortion"]} ./jwst_miri_distortion_new.asdf

We're ready to go about modifying the file.

## Modify the reference file

The next step is to modify the file so that CRDS will accept the update -- CRDS rejects files that it determines to be identical to an existing file.  This is an ASDF file, so we'll use the `asdf` library to open it and peek inside:

In [ ]:
import asdf

af = asdf.open("jwst_miri_distortion_new.asdf", mode="rw")
af.info()

Let's modify something in the "bzero" property:

In [ ]:
asdf.info(af["bzero"])

`beta_zero` seems like a good candidate.  Change that to something we'll recognize later and flush the file to disk:

In [ ]:
af["bzero"]["beta_zero"][0] = -3.14159
af.update()
af.close()

Gee whiz, science is easy!

## Submit the file to CRDS

### Login and navigate to the submission form

Now that we have a modified file, we can submit it to CRDS.  In order to submit files we need to be logged in as a user with the appropriate permissions.  Follow this URL to the home page:

https://jwst-crds-bit.stsci.edu/

and, in the upper-right corner, click the login button:

![Login button](images/login.png)

The login page will ask you to select an instrument to "lock".  This lock grants you exclusive access to that instrument while making your submission.  Note that this means that multiple users cannot complete this notebook simultaneously.

Select "miri" and press `Continue`.  You will be asked to identify yourself with your STScI SSO credentials, and perhaps also to authorize CRDS to access details of your SSO identity.

Once logged in, new links will appear on the home page.  Follow the link titled `Extended Batch Submit References` which should take you to this page:

https://jwst-crds-bit.stsci.edu/submission_form/redcat_submit/

This is the main file submission form.

### Upload the file to the server and fill required form fields

Click the green `Add Files` button and select the `jwst_miri_distortion_new.asdf` file that we modified:

![Add Files button](images/add_files.png)

The next several form fields are for documentation and questions that for an ordinary submission, we'd answer with care.  In this case we'll just fill dummy values in for the required questions so that we can get the form submitted.  In order:

- **Type of files (Bias, Dark, etc.)**: testing
- **Has HISTORY section in the primary...**: yes
- **Has PEDIGREE keyword been checked...**: yes
- **Have REFTYPE and AUTHOR been checked...**: yes
- **Was the DESCRIP keyword updated...**: yes
- **Has the USEAFTER keyword been checked...**: yes
- **Files run through the current version...**: testing
- **Please indicate which modes...**: testing
- **Description of how the files were tested...**: testing

The next form field is `Derive From Context:`.  This tells CRDS what context to start with when adding the modified file.  We should set this to `jwst_0641.pmap`:

![User specified context](images/user_specified_context.png)

There are two more fields that require input:

- **Creator**: testing
- **Reason for Delivery**: testing

Press the blue `Submit References` button and watch the gears turn!

## Inspect and confirm the submission

After the submission has been processed, we arrive at a `Batch Reference Submit Results` page.  The `Certify Results` section shows validation output for each of the submitted files:

![Certify Results](images/certify_results.png)

There are two files?  Didn't we only submit one?  Well, anytime a reference file is submitted, an **.rmap** file should also be submitted.  A reference file without a reference in any **.rmap** is an orphan file; it's impossible for CRDS to reach the file by following its rules.  To save file submitters from having to make rule updates themselves, CRDS by default attempts to generate an updated **.rmap** automatically.  It's done a good job in this case, replacing the old reference file with the new one as expected.  The `Differences` section of the page shows exactly what changed:

```
 selector = Match({
     ('ANY', 'ANY', 'ANY', 'MIR_DARK') : 'N/A',
     ('ANY', 'ANY', 'ANY', 'MIR_FLATIMAGE|MIR_FLAT-IMAGE|MIR_FLATMRS|MIR_FLAT-MRS') : 'N/A',
     ('MIRIFULONG', '34', 'LONG', 'MIR_MRS') : UseAfter({
-        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0032.asdf',
+        '2000-01-01 00:00:00' : 'jwst_miri_distortion_0036.asdf',
     }),
     ('MIRIFULONG', '34', 'MEDIUM', 'MIR_MRS') : UseAfter({
         '2000-01-01 00:00:00' : 'jwst_miri_distortion_0033.asdf',
```

Note that CRDS has also generated a new filename.  Yours will be different from `jwst_miri_distortion_0036.asdf` but will certainly be a later number than the original `0032`.

Since we're happy with this result, we can check the `I confirm that any warnings or differences seen are expected` box at the bottom of the page and press the green `Confirm` button:

![Confirm button](images/confirm_submission.png)

Confirming takes a few seconds and then we arrive at the final result page:

![Final result](images/final_result.png)

CRDS has generated a new context that incorporates your updated **.rmap** and reference file.  In the example above, the new context is `jwst_0642.pmap`, but yours will be different (I've already claimed `jwst_0642.pmap` when writing this notebook).  Find your new context identifier on the page and assign it to the `NEW_CONTEXT` variable below so that we can use its value in subsequent cells.

In [ ]:
NEW_CONTEXT = "your-context-here"
assert NEW_CONTEXT != "your-context-here", "Please set NEW_CONTEXT"

Finally, wait at least 5 minutes to proceed.  CRDS is careful to prohibit access to files that haven't yet been ingested into the archive.  The dev server files aren't archived, but we still need to wait for a cron job to run that will simulate successful ingestion.

## See what we have wrought

Now let's look at the consequences of what we've done.  The `crds` library has some in-memory caches that prevents it from finding our new files.  We need to run some esoteric code to clear those caches and allow it to pick up the new material.  Nothing to see here:

In [ ]:
from itertools import chain
for value in chain(crds.heavy_client.__dict__.values(), crds.api.__dict__.values()):
    if isinstance(value, crds.utils.CachedFunction):
        value.cache = {}

Request the best reference file again:

In [ ]:
crds.getreferences(
    {
        "META.INSTRUMENT.NAME": "MIRI",
        "META.INSTRUMENT.DETECTOR": "MIRIFULONG",
        "META.INSTRUMENT.CHANNEL": "34",
        "META.INSTRUMENT.BAND": "LONG",
        "META.EXPOSURE.TYPE": "MIR_MRS",
        "META.OBSERVATION.DATE": "2014-11-20",
        "META.OBSERVATION.TIME": "11:53:02",
    },
    reftypes=["distortion"],
    observatory="jwst",
    context="jwst_0641.pmap",
)

Er, wait, that's the same file.  As well it should be!  I just wanted to emphasize the immutability of contexts.  Even though we've delivered a new reference file, `crds.getreferences` returns the original file _because we haven't changed the context parameter_.  This magnificent feature is what allows a user to work with a frozen set of reference files (and rules) even while file submissions are proceeding in the background.  When we request our new context, we see the updated file:

In [ ]:
result = crds.getreferences(
    {
        "META.INSTRUMENT.NAME": "MIRI",
        "META.INSTRUMENT.DETECTOR": "MIRIFULONG",
        "META.INSTRUMENT.CHANNEL": "34",
        "META.INSTRUMENT.BAND": "LONG",
        "META.EXPOSURE.TYPE": "MIR_MRS",
        "META.OBSERVATION.DATE": "2014-11-20",
        "META.OBSERVATION.TIME": "11:53:02",
    },
    reftypes=["distortion"],
    observatory="jwst",
    context=NEW_CONTEXT,
)
result

There it is!  Let's open the file and make sure it contains our modified `bzero` value:

In [ ]:
af = asdf.open(result["distortion"])
asdf.info(af["bzero"])

There's our old friend, negative-pi.  Hooray!

In [ ]:
# Cleanup
af.close()

## Further reading

The CRDS User Manual includes [detailed documentation](https://jwst-crds.stsci.edu/static/users_guide/file_submissions.html) on use of the web submission form.